In [ ]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime
from helper_functions import create_df, updated_data_pull, upsert, delete_record, insert_records

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


#### need to develop config file, need to develop a script to validate input data and output data, logging any errors put in review area and handle efficiently, look into scheduling, report generation of data... this will also be a new sf object




Source_data = pd.read_csv("/mnt/c/Users/ralvin/Downloads/837I00111182024.csv")



Provider_TIN_dict = {'MAPPINGS':{'TIN':['Name', 'Unique_TIN__c'], 'Address':['Address__c'], 'City':['City__c'], 'State':['State__c'], 'Zip':['Zip__c'],
                      'Provider':['Provider_Name__c']}, 'KEYS':{'Provider_TIN__c':'Unique_TIN__c'}}

Account_dict = {'MAPPINGS':{'TIN':['Tax_ID__c'], 'BillProvAddress1':['BillingAddress'], 'BillProvCity':['BillingCity'], 'BillProvState':['BillingState'],
                 'BillProvZip':['BillingPostalCode'], 'Provider':['Name']}, 'KEYS':{'Account':'Tax_ID__c'}}



Claim_dict = {'MAPPINGS':{'Claim ID':['Claim_ID__c'], 'TIN':['Provider_TIN__c'], 'Patient Group/Policy Number':['Group_Client__c'], 'Claim Number':['Claim_Number__c'],'JurisdictionState':['Jurisdiction__c'],
               'Diag Code 1st':['Diag_Code__c'],'Diag Code 2nd':['Diag_Code_2nd__c'], 'Diag Code 3rd':['Diag_Code_3rd__c'], 'Diag Code 4th':['Diag_Code_4th__c'], 'Provider':['Provider__c'],
               'Patient':['Patient__c'], 'Patient ID':['Patient_ID__c'], 'Date of Birth':['Date_of_Birth__c'],'Zip':['Provider_Zip__c'], 'DRG':['DRG__c'], 'IsQPA':['QPA__c'], 'NPI':['NPI__c'],
               'PlaceofService':['POS__c'], 'Billing Provider Taxonomy':['Provider_Specialty__c', 'Service_Type__c']}, 'KEYS':{'Claims__c':'Claim_ID__c'}, 
               'TABLE_CONVERT':{'TIN':{'Provider_TIN__c':['Name', 'Id']}, 'Patient Group/Policy Number':{'Groups_Clients__c':['Group_Number__c', 'Id']}, 'JurisdictionState':{'Jurisdiction__c':['Name', 'Id']},
                                 'DRG':{'DRG__c':['Name', 'Id']}, 'Billing Provider Taxonomy':{'Provider_Specialty__c':['Name', 'Id', 'Service_Type__c']}},
                'KEYS':{'Claims__c':'Claim_ID__c'}
               }

Line_item_dict = {'MAPPINGS':{'Begin DOS':['DOS__c'], 'Revenue Code':['Rev_Code__c'],'Billed Amount':['Billed_Amount__c'], 'Claim ID':['RHP_ID__c'], 'Mod':['Mod__c'],
                   'Mod (2nd)':['Mod_2nd__c'], 'Units':['Units__c'], 'MAR':['MAR__c'], 'HCPCS/CPT Code':['HCPCS_CPT_Code__c'], 'Line ID':['Line_ID__c'], 
                    '4LCodes':['Exp_Code_2nd__c']},
                  'KEYS':{'Line_Items__c':'RHP_ID__c'},
                  'TABLE_CONVERT':{'Claim ID':{'Claims__c':['Claim_ID__c', 'Id']}, 'HCPCS/CPT Code':{'HCPCS_CPT_Code__c':['Name', 'Id']}}
                  }



def salesforce_connection(sandbox=True):
    session = requests.Session()
    security_token = 'pIjwc8P0OB2ZisbQYhlKxfLd'
    username = 'ralvin@relianthp.com.tctdev'
    password = 'Flash363!'
    domain = 'test'

    session_id, instance = SalesforceLogin(username=username, password=password, security_token=security_token, domain=domain)
    sf = Salesforce(instance=instance, session_id=session_id, session=session)
    print('Connected to Salesforce (Sandbox)')
    return sf

sf = salesforce_connection()

all_objects = sf.describe()["sobjects"]


accounts_metadata = sf.Account.describe()
account_cols = [field['name'] for field in accounts_metadata['fields']]

provider_tin_metadata = sf.Provider_TIN__c.describe()
provider_tin_cols = [field['name'] for field in provider_tin_metadata['fields']]

claims_metadata = sf.Claims__c.describe()
claim_cols = [field['name'] for field in claims_metadata['fields']]

line_item_metadata = sf.Line_Items__c.describe()
line_item_cols = [field['name'] for field in line_item_metadata['fields']]

clients_metadata = sf.Clients__c.describe()
clients_cols = [field['name'] for field in clients_metadata['fields']]

group_clients_metadata = sf.Groups_Clients__c.describe()
group_clients_cols = [field['name'] for field in group_clients_metadata['fields']]

jurisdiction_metadata = sf.Jurisdiction__c.describe()
jurisdiction_cols = [field['name'] for field in jurisdiction_metadata['fields']]

drg_metadata = sf.DRG__c.describe()
drg_cols = [field['name'] for field in drg_metadata['fields']]

hcpcs_cpt_metadata = sf.HCPCS_CPT_Code__c.describe()
hcpcs_cpt_cols = [field['name'] for field in hcpcs_cpt_metadata['fields']]

provider_specialty_metadata = sf.Provider_Specialty__c.describe()
provider_specialty_cols = [field['name'] for field in provider_specialty_metadata['fields']]

needed_objects = { "Claims__c": claim_cols, "Line_Items__c": line_item_cols, "Provider_TIN__c": provider_tin_cols, 
                 'Jurisdiction__c': jurisdiction_cols, 'Groups_Clients__c': group_clients_cols, 'DRG__c': drg_cols, 
                 'HCPCS_CPT_Code__c': hcpcs_cpt_cols, 'Account':account_cols, 'Clients__c': clients_cols, 'Provider_Specialty__c':provider_specialty_cols}


DATA_MAPPINGS = {'Provider_Object_Map':Provider_TIN_dict, 'Account_Object_Map':Account_dict, 'Claim_Object_Map':Claim_dict, 'Line_Item_Object_Map':Line_item_dict}




table_store = {}

for object_name in needed_objects:
    print(object_name)

    object_fields = needed_objects[object_name]

    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)
    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')
    print(df.shape)
    table_store[object_name] = df

Connected to Salesforce (Sandbox)
Claims__c
(12937, 100)
Line_Items__c
(22841, 48)
Provider_TIN__c
(10857, 27)
Jurisdiction__c
(53, 16)
Groups_Clients__c
(1703, 58)
DRG__c
(808, 20)
HCPCS_CPT_Code__c
(10047, 36)
Account
(2217, 98)
Clients__c
(63, 58)
Provider_Specialty__c
(870, 18)


In [2]:
table_store['Account'].head()

,Id,IsDeleted,MasterRecordId,Name,Type,RecordTypeId,ParentId,BillingStreet,BillingCity,BillingState,BillingPostalCode,BillingCountry,BillingStateCode,BillingCountryCode,BillingLatitude,BillingLongitude,BillingGeocodeAccuracy,BillingAddress,ShippingStreet,ShippingCity,ShippingState,ShippingPostalCode,ShippingCountry,ShippingStateCode,ShippingCountryCode,ShippingLatitude,ShippingLongitude,ShippingGeocodeAccuracy,ShippingAddress,Phone,Fax,Website,PhotoUrl,Industry,AnnualRevenue,NumberOfEmployees,TickerSymbol,Description,OwnerId,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,IsPartner,IsCustomerPortal,Jigsaw,JigsawCompanyId,AccountSource,SicDesc,Copy_Billing_Address_to_Shipping_Address__c,rsdoc__RS_Document_Error__c,rsdoc__RS_Document_to_Merge__c,rh2__testCurrency__c,Insurance_Type__c,Claims_System__c,dba_fka__c,Account_Acronym__c,Status__c,Tax_ID__c,NPI_Number__c,External_ID__c,Multiple_Locations__c,Special_Pricing__c,Special_Pricing_Notes__c,Email__c,Billing_Address__Street__s,Billing_Address__City__s,Billing_Address__PostalCode__s,Billing_Address__StateCode__s,Billing_Address__CountryCode__s,Billing_Address__Latitude__s,Billing_Address__Longitude__s,Billing_Address__GeocodeAccuracy__s,Billing_Address__c,Rendering_Address__Street__s,Rendering_Address__City__s,Rendering_Address__PostalCode__s,Rendering_Address__StateCode__s,Rendering_Address__CountryCode__s,Rendering_Address__Latitude__s,Rendering_Address__Longitude__s,Rendering_Address__GeocodeAccuracy__s,Rendering_Address__c,Service_Address__Street__s,Service_Address__City__s,Service_Address__PostalCode__s,Service_Address__StateCode__s,Service_Address__CountryCode__s,Service_Address__Latitude__s,Service_Address__Longitude__s,Service_Address__GeocodeAccuracy__s,Service_Address__c,Integration_Setting_Name__c,Default_Pricing_Method__c
0,0018a00002838fXAAQ,False,None,Brown & Brown Insurance,None,0128a000000ijxDAAQ,0018a0000283Ak8AAE,1001 Mark Avenue\r\nSuite 201,Carpinteria,California,93013,United States,CA,US,None,None,None,"{'city': 'Carpinteria', 'country': 'United Sta...",1001 Mark Avenue\r\nSuite 201,Carpinteria,California,93013,United States,CA,US,None,None,None,"{'city': 'Carpinteria', 'country': 'United Sta...",8059650071,None,bbinsurance.com,/services/images/photo/0018a00002838fXAAQ,None,NaN,NaN,NYSE: BRO,None,0058a00000M3ZigAAF,2023-03-09T16:14:02.000+0000,0058a00000M3ZigAAF,2023-09-24T16:17:25.000+0000,0058a00000M3ZigAAF,2023-09-24T16:17:25.000+0000,None,None,None,False,False,None,None,Advertisement,None,True,None,None,None,All,None,None,None,Prospect,None,None,None,False,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0018a0000283Ak8AAE,False,None,"Brown & Brown, Inc.",None,0128a000000ijxDAAQ,None,300 North Beach Street,Daytona Beach,Florida,32114,United States,FL,US,None,None,None,"{'city': 'Daytona Beach', 'country': 'United S...",None,None,None,None,None,None,None,None,None,None,None,3862529601,None,bbinsurance.com,/services/images/photo/0018a0000283Ak8AAE,None,NaN,NaN,BRO,None,0058a00000M3ZigAAF,2023-03-09T17:05:44.000+0000,0058a00000M3ZigAAF,2023-03-09T17:05:44.000+0000,0058a00000M3ZigAAF,2023-03-12T23:50:08.000+0000,None,None,None,False,False,None,None,Advertisement,None,False,None,None,None,All,None,None,None,None,None,None,None,False,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,0018a0000283P1NAAU,False,None,"Zelis Healthcare, Inc.",Competitor,0128a000000ijxSAAQ,None,2 Crossroads Drive,Bedminster,New Jersey,07921,United States,NJ,US,None,None,None,"{'city': 'Bedminster', 'country': 'United Stat...",None,None,None,None,None,None,None,None,None,None,None,8883113505,None,zelis.com,/services/images/photo/0018a0000283P1NAAU,None,NaN,NaN,None,None,0058a00000M3ZigAA

In [ ]:
###claims starting count is 12,931
### Line_Items__c starting count is 22,789

In [2]:
Povider_map = DATA_MAPPINGS['Provider_Object_Map']
Account_map = DATA_MAPPINGS["Account_Object_Map"]
Claims_map = DATA_MAPPINGS['Claim_Object_Map']



#### Note that records being inserted with the same data are created and have the same Id in the claims table


map_account_table, _ = create_df(Account_map, Source_data, needed_objects, sf)
map_provider_table, keys = create_df(Povider_map, Source_data, needed_objects, sf)
### upsert provider data before populating claims table
update_provider_object = upsert(map_provider_table, keys, sf)
map_claims_table, keys = create_df(Claims_map, Source_data, needed_objects, sf)



{'Provider_TIN__c': ['Name', 'Id']}
pulling updated Provider_TIN__c table
Skipping column Name or no target column available.
Mapping Id to Provider_TIN__c
{'Groups_Clients__c': ['Group_Number__c', 'Id']}
pulling updated Groups_Clients__c table
Skipping column Group_Number__c or no target column available.
Mapping Id to Group_Client__c
{'Jurisdiction__c': ['Name', 'Id']}
pulling updated Jurisdiction__c table
Skipping column Name or no target column available.
Mapping Id to Jurisdiction__c
{'DRG__c': ['Name', 'Id']}
pulling updated DRG__c table
Skipping column Name or no target column available.
Mapping Id to DRG__c
{'Provider_Specialty__c': ['Name', 'Id', 'Service_Type__c']}
pulling updated Provider_Specialty__c table
Skipping column Name or no target column available.
Mapping Id to Provider_Specialty__c
Mapping Service_Type__c to Service_Type__c


In [8]:
#### insert claim records into sf object
new_claim_ids, error_logs = insert_records(map_claims_table.drop_duplicates(), keys, sf)

Processing record 0: {'Claim_ID__c': 'A3038B5E4B6C914tksft', 'Provider_TIN__c': 'a0FVF000001zh5t2AA', 'Group_Client__c': 'a0C8a00000r3GJ6EAM', 'Claim_Number__c': '4.32E+14', 'Jurisdiction__c': 'a024W00000ICbumQAD', 'Diag_Code__c': 'M25562', 'Diag_Code_2nd__c': nan, 'Diag_Code_3rd__c': nan, 'Diag_Code_4th__c': nan, 'Provider__c': 'KAISER FOUNDATION HOSPITALS ', 'Patient__c': 'WATKINS MONIC  ', 'Patient_ID__c': '569713835', 'Date_of_Birth__c': '8/12/1982', 'Provider_Zip__c': 900749998, 'DRG__c': nan, 'QPA__c': 'N', 'NPI__c': nan, 'POS__c': 13, 'Provider_Specialty__c': 'a008a000015yXODAA2', 'Service_Type__c': 'OUTPATIENT'}
Processing record 1: {'Claim_ID__c': 'BE9124A610A9998tksft', 'Provider_TIN__c': 'a0F4W00000W1KI4UAN', 'Group_Client__c': 'a0C8a00000sg4tZEAQ', 'Claim_Number__c': 'CLU20241115325204', 'Jurisdiction__c': 'a024W00000HYAjJQAX', 'Diag_Code__c': 'M7672', 'Diag_Code_2nd__c': 'M722', 'Diag_Code_3rd__c': 'M7732', 'Diag_Code_4th__c': nan, 'Provider__c': 'JOHNSON MEMORIAL HOSPITAL

In [10]:
error_logs

[simple_salesforce.exceptions.SalesforceMalformedRequest('https://relianthealthpartners2020--tctdev.sandbox.my.salesforce.com/services/data/v59.0/sobjects/Claims__c/',
                                                         400,
                                                         'Claims__c',
                                                         [{'message': 'Required fields are missing: [Group_Client__c]',
                                                           'errorCode': 'REQUIRED_FIELD_MISSING',
                                                           'fields': ['Group_Client__c']}]),
 simple_salesforce.exceptions.SalesforceMalformedRequest('https://relianthealthpartners2020--tctdev.sandbox.my.salesforce.com/services/data/v59.0/sobjects/Claims__c/',
                                                         400,
                                                         'Claims__c',
                                                         [{'message': 'Required fields ar

In [11]:
#### repull updated claims data

claims_table_repulled = updated_data_pull('Claims__c', needed_objects, sf)
claims_table_repulled.shape

(13033, 100)

In [16]:
#### create the lines table for sf ingestion 
Lines_map = DATA_MAPPINGS['Line_Item_Object_Map']
map_lines_table, keys = create_df(Lines_map, Source_data, needed_objects, sf)
map_lines_table.shape

{'Claims__c': ['Claim_ID__c', 'Id']}
pulling updated Claims__c table
Skipping column Claim_ID__c or no target column available.
Mapping Id to RHP_ID__c
{'HCPCS_CPT_Code__c': ['Name', 'Id']}
pulling updated HCPCS_CPT_Code__c table
Skipping column Name or no target column available.
Mapping Id to HCPCS_CPT_Code__c


(446, 11)

In [22]:
##### insert the lines data onto salesforce 
new_lines_ids = insert_records(map_lines_table, keys, sf)




Processing record 0: {'DOS__c': '9/26/2024', 'Rev_Code__c': 320, 'Billed_Amount__c': 987.0, 'RHP_ID__c': 'a06VF00000A1117YAB', 'Mod__c': nan, 'Mod_2nd__c': nan, 'Units__c': 1, 'MAR__c': 156.72, 'HCPCS_CPT_Code__c': 'a0b4W00000y1IFrQAM', 'Line_ID__c': 1, 'Exp_Code_2nd__c': nan}
Processing record 1: {'DOS__c': '10/17/2024', 'Rev_Code__c': 510, 'Billed_Amount__c': 65.0, 'RHP_ID__c': 'a06VF00000A112jYAB', 'Mod__c': nan, 'Mod_2nd__c': nan, 'Units__c': 1, 'MAR__c': 65.0, 'HCPCS_CPT_Code__c': 'a0b4W00000y1JRaQAM', 'Line_ID__c': 1, 'Exp_Code_2nd__c': nan}
Processing record 2: {'DOS__c': '10/22/2024', 'Rev_Code__c': 278, 'Billed_Amount__c': 3.0, 'RHP_ID__c': 'a06VF00000A114LYAR', 'Mod__c': nan, 'Mod_2nd__c': nan, 'Units__c': 1, 'MAR__c': 3.0, 'HCPCS_CPT_Code__c': nan, 'Line_ID__c': 1, 'Exp_Code_2nd__c': nan}
Processing record 3: {'DOS__c': '10/22/2024', 'Rev_Code__c': 300, 'Billed_Amount__c': 54.0, 'RHP_ID__c': 'a06VF00000A114LYAR', 'Mod__c': nan, 'Mod_2nd__c': nan, 'Units__c': 1, 'MAR__c': 54.

In [33]:
# new_claims = claims_table_repulled[claims_table_repulled['Id'].isin(new_claim_ids)]
# print(new_claims.shape)
# # new_claims.head()

# lines = table_store['Line_Items__c']
# lines[lines['RHP_ID__c'].isin(new_claim_ids)]


In [31]:
map_claims_table[map_claims_table['Group_Client__c'].isna()].shape ##### these are the cases where lines table insert failed

(22, 20)

In [34]:
#### repull updated lines data

lines_table_repulled = updated_data_pull('Line_Items__c', needed_objects, sf)
lines_table_repulled.shape

(23213, 48)

In [44]:
# lines_table_repulled[lines_table_repulled['RHP_ID__c'] == '1081192']

claims_table_repulled[claims_table_repulled['Claim_ID__c'] == '317D671330C4442tksft']

,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Duplicate__c,Client__c,Claim_Number__c,Jurisdiction__c,Provider_TIN__c,Contains_Line_Items__c,Status__c,Appeal__c,Date_Fee_Paid__c,Fee_Credit_Date__c,Diag_Code__c,Diag_Code_2nd__c,Diag_Code_3rd__c,Proc_Code__c,Proc_Code_2nd__c,Diag_Code_4th__c,Total_Billed__c,Provider__c,Group_Client__c,Claim_ID__c,Received_Date__c,Appeal_Rec_d_Date__c,Last_Contact__c,Appeal_Stage__c,State_Appeal__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Appeal_Closed_Date__c,POS__c,Attorney_Assigned__c,Detailed_Appeal_Resp__c,Begin_DOS__c,End_DOS__c,Total_MAR__c,Total_Allowed__c,Total_Reductions__c,Fee__c,Patient__c,Date_of_Birth__c,Patient_ID__c,Total_Additional_Payment__c,Fee_Credit__c,Savings__c,Provider_Zip__c,DRG__c,Pended_Token__c,Client_Id__c,Service_Type__c,PHS_Id__c,Total_Reconsidered__c,X3_Digit_Zip__c,Prime_Return_Filename__c,Contact_Name__c,Proposed_Settlement__c,Proposed_Additional__c,Paid_Fee__c,Gross_Fee__c,QPA__c,Submitted__c,Provider_Specialty__c,Exp_Code_Test__c,Exp_Code_Test2__c,Pricing_Method__c,PEPM_Fee__c,Total_QPA_Allowed__c,NPI__c,SLA__c,Created_Date_Formula__c,Balance_Bill__c,Contract__c,Producer__c,Payer__c,Company__c,API_Status__c,API_Confirmation__c,Total_Savings__c,Bill_Type__c,Form_Type__c,Provider_Account__c,Original_Claim_for_Reconsideration__c,Return_Date_Deadline__c,Savings_Threshold__c,Total_Code_Edit__c,Original_Claim_from_Duplication__c,Recon_Dupe__c,Blank_Allowed_Count__c
12960,a06VF00000A114LYAR,False,1081131,2024-12-16T20:51:32.000+0000,005VF00000AhMNjYAN,2024-12-16T20:51:32.000+0000,005VF00000AhMNjYAN,2024-12-16T20:51:32.000+0000,None,2024-12-16T20:51:32.000+0000,2024-12-16T20:51:32.000+0000,1984-03-126104617530,a048a00000iOnmHAAS,U424319281216,a024W00000HYAjcQAH,a0FVF000001f3W22AI,False,To be Repriced,False,None,None,Z1211,K635,K648,None,None,K2950,0.0,DEACONESS HENDERSON HOSPII,a0CUu000001thoWMAQ,317D671330C4442tksft,2024-12-16,None,None,None,False,None,None,None,None,None,None,13,False,None,None,None,0.0,0.0,0.0,0.0,COBB BETHAN,1984-03-12,FBDS11454,0.0,0.0,NaN,452638704,None,None,None,OUTPATIENT,None,0.0,452,REL_,None,NaN,0.0,None,0.0,N,None,a008a000015yXODAA2,None,None,FMP,NaN,None,1295211563,NaN,2024-12-16T20:51:32.000+0000,False,None,None,None,None,None,None,NaN,None,HCFA,None,None,None,NaN,0.0,None,None,0.0


In [52]:
lines_table_repulled[lines_table_repulled['RHP_ID__c'] == 'a06VF00000A114LYAR']

,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastViewedDate,LastReferencedDate,DOS__c,Rev_Code__c,Line_Number__c,Billed_Amount__c,RHP_ID__c,Mod__c,Mod_2nd__c,Units__c,MAR__c,Allowed__c,Client__c,Reconsideration__c,Savings__c,Reductions__c,Bill_Type__c,Exp_Desc__c,HCPCS_CPT_Code__c,Additional_Payment__c,Line_ID__c,Pricing_Method__c,Provider_TIN_Claim__c,Jurisdiction_Claim__c,Pricing_Trigger__c,Exp_Code__c,Exp_Code_2nd__c,MO_QPA_Allowed__c,Exp_Desc_2nd__c,MO_QPA_Reductions__c,MO_Fac_QPA_Reductions__c,MO_Fac_QPA_Allowed__c,MO_Anes_QPA__c,QPA_Allowed__c,MO_Anes_QPA_Reductions__c,QPA_Reductions__c,Unique_Line_Id__c,Code_Edit__c,NDC_Number__c,Target_Amount__c
22797,a0YVF0000012lLd2AI,False,L2307837,2024-12-16T21:06:17.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:17.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:17.000+0000,None,None,2024-10-22,278,None,3.0,a06VF00000A114LYAR,None,None,1.0,3.0,NaN,a048a00000iOnmH,None,100.0,3.0,None,None,None,None,1.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,0.00,None,3.00,3.00,0.00,0.0,NaN,3.0,3.0,None,NaN,None,0.0
22799,a0YVF0000012lNF2AY,False,L2307838,2024-12-16T21:06:18.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:18.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:18.000+0000,None,None,2024-10-22,300,None,54.0,a06VF00000A114LYAR,None,None,1.0,54.0,NaN,a048a00000iOnmH,None,100.0,54.0,None,None,a0b4W00000y1IqZQAU,None,2.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,7.84,None,46.16,-6.80,60.80,0.0,NaN,54.0,54.0,None,NaN,None,0.0
22800,a0YVF0000012lOr2AI,False,L2307839,2024-12-16T21:06:18.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:18.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:18.000+0000,None,None,2024-10-22,310,None,1000.0,a06VF00000A114LYAR,59,None,4.0,1000.0,NaN,a048a00000iOnmH,None,100.0,1000.0,None,None,a0b4W00000y1INfQAM,None,3.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,394.24,None,605.76,594.44,405.56,0.0,NaN,1000.0,1000.0,None,NaN,None,0.0
22804,a0YVF0000012lQT2AY,False,L2307840,2024-12-16T21:06:19.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:19.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:19.000+0000,None,None,2024-10-22,310,None,1159.0,a06VF00000A114LYAR,None,None,1.0,1159.0,NaN,a048a00000iOnmH,None,100.0,1159.0,None,None,a0b4W00000y1INwQAM,None,4.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,95.43,None,1063.57,1003.35,155.65,0.0,NaN,1159.0,1159.0,None,NaN,None,0.0
22805,a0YVF0000012lS52AI,False,L2307841,2024-12-16T21:06:19.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:19.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:19.000+0000,None,None,2024-10-22,370,None,1227.0,a06VF00000A114LYAR,None,None,3.0,1227.0,NaN,a048a00000iOnmH,None,100.0,1227.0,None,None,None,None,5.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,0.00,None,1227.00,1227.00,0.00,0.0,NaN,1227.0,1227.0,None,NaN,None,0.0
22812,a0YVF0000012lTh2AI,False,L2307842,2024-12-16T21:06:19.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:19.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:19.000+0000,None,None,2024-10-22,636,None,155.0,a06VF00000A114LYAR,None,None,70.0,155.0,NaN,a048a00000iOnmH,None,100.0,155.0,None,None,a0b4W00000y1JZ9QAM,None,6.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,9.10,None,145.90,155.00,0.00,0.0,NaN,155.0,155.0,None,NaN,None,0.0
22818,a0YVF0000012lVJ2AY,False,L2307843,2024-12-16T21:06:20.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:20.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:20.000+0000,None,None,2024-10-22,710,None,48.0,a06VF00000A114LYAR,None,None,3.0,48.0,NaN,a048a00000iOnmH,None,100.0,48.0,None,None,None,None,7.0,FMP,a0FVF000001f3W22AI,a024W00000HYAjc,False,None,None,0.00,None,48.00,48.00,0.00,0.0,NaN,48.0,48.0,None,NaN,None,0.0
22819,a0YVF0000012lVK2AY,False,L2307845,2024-12-16T21:06:21.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:21.000+0000,005VF00000AhMNjYAN,2024-12-16T21:06:21.000+0000,None,None,2024-10-22,750,None,3326.0,a06VF00000A114LYAR,PT,None,1.0,3326.0,NaN,a048a00000iOnmH,None,100.0,3326.

In [51]:
lines_table_repulled[lines_table_repulled['RHP_ID__c'].isin(new_claim_ids)].shape

(424, 48)

In [84]:

###### seeing duplicate TIN rows with varying info... will need to solve how we ingest this into providers table

# Povider_map = DATA_MAPPINGS['Provider_Object_Map']


# map_provider_table, keys = create_df(Povider_map, Source_data, table_store)
# df = map_provider_table.drop_duplicates()
# duplicate_rows = df[df.duplicated(subset=['Unique_TIN__c'], keep=False)]
# duplicate_rows  




In [52]:
provider_table = table_store['Provider_TIN__c']
provider_table.shape

(10853, 27)

In [53]:
unique_providers = [str(i) for i in list(set((map_provider_table.drop_duplicates()['Unique_TIN__c'])))]
len(unique_providers)

59

In [66]:
temp = provider_table[provider_table['Unique_TIN__c'].isin(unique_providers)]
my_provider_ids = list(temp['Id'])

len(my_provider_ids)

59

In [109]:
delete_record(map_claims_table, keys, table_store, sf)

pulling updated Claims__c table
Object: Claims__c, Records to process: 446
Matching record IDs to delete: 102
All records deleted successfully.
